<a href="https://colab.research.google.com/github/jinweiRao/MEng-Project/blob/master/Test_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow
from tensorflow import keras
from keras.optimizers import SGD, Adam,RMSprop
from keras.layers import Dense,Activation,Input
import numpy as np
import pandas as pd
from keras import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from keras import optimizers, regularizers
import seaborn as sns
from keras import callbacks
from scipy.stats.stats import pearsonr
import random
import matplotlib.pyplot
from sklearn.cluster import KMeans
from sklearn import tree
from tqdm import tqdm

## Build baseline_model

In [0]:
data=pd.read_csv('plant_A.csv')

In [3]:
data1 = data.copy()
data1.Temp,data1.CaO=data1.Temp/1000,data1.CaO/10
data1.head()

,lp,Temp,CaO,MgO,SiO2,Fe_total,MnO,Al2O3,TiO2,V2O5
0,4.736198,1.649,4.00,8.63,7.8,27.5,3.38,0.91,0.45,1.473
1,4.436094,1.654,3.96,9.99,9.2,23.0,4.14,2.60,0.68,1.945
2,4.499810,1.664,4.48,8.80,11.0,20.2,3.45,1.58,0.76,2.070
3,4.140865,1.680,4.24,8.10,11.5,21.1,3.76,2.47,0.88,2.161
4,4.081441,1.671,4.88,11.73,11.8,13.3,3.26,1.48,0.78,1.976


In [0]:
def Mse(prediction,truth):
  mse = sum((prediction - truth)**2)/len(truth)
  return mse

In [0]:
data1['Basicity'] = (data['CaO'] + 0.3*data['MgO'])/data['SiO2']
data['Basicity'] = (data['CaO'] + 0.3*data['MgO'])/data['SiO2']

In [0]:
X_tree = data.loc[:,'Temp':'Basicity':9]
y_tree = data.loc[:,'lp']

In [0]:
X = data1.drop(['lp','Basicity'],axis=1).values
y = data1.lp.values
#X_train,X_test=X[:8000],X[8001:11000],X[11001:]
#y_train,y_test=y[:8000],y[8001:11000],y[11001:]

In [0]:
clf = tree.DecisionTreeRegressor(max_depth = 2, criterion = 'mse')
clf = clf.fit(X_tree,y_tree)

In [9]:
tree_result = tree.export_text(clf,feature_names = ['Temp','Basicity'])
print(tree_result)

|--- Temp <= 1653.50
|   |--- Basicity <= 3.19
|   |   |--- value: [4.26]
|   |--- Basicity >  3.19
|   |   |--- value: [4.44]
|--- Temp >  1653.50
|   |--- Temp <= 1663.50
|   |   |--- value: [4.23]
|   |--- Temp >  1663.50
|   |   |--- value: [4.12]



In [0]:
act_list = ['sigmoid','tanh','softsign']
opt_list = ['adam','RMSprop','Adadelta','SGD','Nadam']

In [0]:
def baseline_model(act,opt):
    model = Sequential()
    model.add(Dense(20, input_dim = 9, activation = act))
    model.add(Dense(1, activation = 'linear'))
    model.compile(loss='mse', optimizer = opt, metrics=['mse']) 
    return model

In [0]:
def generate_model():
  model_list = []
  for act in act_list:
    for opt in opt_list:
      model_list.append(baseline_model(act,opt))
  return model_list

In [0]:
modelList = generate_model()

In [0]:
T1 = data1.loc[data1['Temp']<=1.653].copy()
T2 = data1.loc[data1['Temp']>1.653].copy()
B1 = T1.loc[T1['Basicity'] <= 3.19].reset_index().drop(['index'],axis = 1)
B2 = T1.loc[T1['Basicity'] > 3.19].reset_index().drop(['index'],axis = 1)
B3 = T2.loc[T2['Basicity'] <= 3.46].reset_index().drop(['index'],axis = 1)
B4 = T2.loc[T2['Basicity'] > 3.46].reset_index().drop(['index'],axis = 1)
  

In [0]:
def cv(df,model):
  kf = KFold(n_splits = 5, random_state = 10, shuffle = False)
  splits = kf.split(df)
  comparision_list = []
  for train_index,test_index in tqdm(splits):
    X_train = df.loc[train_index,'Temp':'V2O5'].values
    X_test = df.loc[test_index,'Temp':'V2O5'].values
    y_train = df.loc[train_index,'lp'].values
    y_test = df.loc[test_index,'lp'].values
    model.fit(X_train, y_train, epochs = 150, batch_size = 20, validation_data = (X_test,y_test),verbose = 0)
    prediction = model.predict(X_test).reshape(len(X_test),)
    comparision = {'prediction':prediction,'truth':y_test}
    comparision_list.append(comparision)
  return comparision_list

In [0]:
def modelsearch(modelList,df):
  result_list = [] 
  for model in tqdm(modelList):
    result = cv(df,model)
    result_list.append(result)
  return result_list

In [0]:
def allsearch():
  final_list = []
  for df in [B1,B2,B3,B4]:
    final = modelsearch(modelList,df)
    final_list.append(final)
  return final_list

In [0]:
B1_result = modelsearch(modelList,B1)